In [1]:
import pandas as pd
import numpy as np
import glob
import os
import copy

In [2]:
def readDir():
    """
    Returns a dataframe of the data files in the directory with the bikeshare data.
    """
    data = pd.DataFrame()
    
    path = os.path.join(os.sep, "raid", "public_data", "Bike_Share_Data", "Boston")
    
    for filename in glob.glob(os.path.join(path, '*.csv')):
        # Get the filesizes in megabytes
        f_size = os.path.getsize(filename) / 1_000_000
        
        # A dictionary stores information about the files to be be put in the df
        file = {"Filename": filename, "Size (mb)": f_size}
        data = data.append(file, ignore_index=True)
        
    return data

def openCSV(files, index):
    """
    Using filenames retrieved with readDir, use this function to open a specific
    csv. Returns a dataframe with the data requested.
    """
    data = pd.read_csv(files.loc[index, 'Filename'])
    if index != 0:
        data['starttime'] = pd.to_datetime(data['starttime'])
        data['stoptime'] = pd.to_datetime(data['stoptime'])
#         files.loc[index, 'Month'] = data.loc[0, 'starttime'].month
#         files.loc[index, 'Year']  = data.loc[0, 'starttime'].year
    return data
    

In [4]:
files = readDir()

data = []

for i in range(files.index.values.shape[0]):
    data.append(openCSV(files, i))
    
fulldf = pd.concat(data[1:])

In [11]:
fulldf["birth year"].unique()

array(['1990', '1985', '1988', '1986', '1991', '1993', '1992', '1970',
       '1981', '1982', '1980', '1974', '1966', '1976', '1973', '1978',
       '1987', '1975', '1989', '1955', '1965', '1967', '1962', '1979',
       '1983', '1958', '1977', '1971', '1968', '1994', '1960', '1950',
       '1945', '1961', '1998', '1984', '1969', '1957', '1963', '1996',
       '1997', '1964', '1995', '1956', '\\N', '1954', '1959', '1944',
       '1953', '1972', '1946', '1952', '1999', '1948', '1947', '1937',
       '1917', '1942', '1943', '1949', '1933', '1934', '1951', '1941',
       '2000', '1939', '1902', '1886', '1900', '2001', '1897', '1940',
       '1899', '1901', '1932', '1938', '1927', 1994, 1991, 1987, 1969,
       1982, 1993, 1985, 1976, 1984, 1988, 1974, 1953, 1992, 1990, 1962,
       1980, 1961, 1955, 1956, 1989, 1965, 1960, 1958, 1959, 1981, 1973,
       1986, 1957, 1983, 1968, 1996, 1967, 1998, 1978, 1952, 1970, 1971,
       1972, 1954, 1966, 1977, 1963, 1975, 1934, 1950, 1979, 1964, 1948,